<a href="https://colab.research.google.com/github/jopapo/nuinvest_options_irpf/blob/main/A%C3%A7%C3%B5es_NuInvest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Not installed libs

!pip install pdfminer

In [31]:
# Requirements

import requests
from pathlib import Path
from getpass import getpass

In [26]:
# Variables
year = 2021

# You should login to your nuinvest account and get the bearer token from the request
access_token = getpass('bearer token')

session = requests.Session()
session.headers.update({'Authorization': f"Bearer {access_token}"})


In [27]:
# Get Invoices Pdfs

def get_invoices(prefix):

  response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/1/{prefix}?startDate={year}-01-01&endDate={year}-12-31")
  response.raise_for_status()

  print(prefix, response)

  invoices = []
  if response.status_code == 204:
    print(f'No data for {prefix}')
  else:
    invoices = response.json()['value']['invoices']
  
  for invoice in invoices:
    params_values = {key:val for (key,val) in invoice.items() if key in ['invoiceNumber', 'custodyId', 'date']}
    # FI não tem custodyId
    # RV tem o data e não tem custodyid
    # Não testado com TP e PS pq eu não tinha papéis pra isso

    response = session.get(f"https://www.nuinvest.com.br/api/gringott/invoices/report/1/{prefix}", params=params_values, stream=True)
    response.raise_for_status()

    filename = Path(f"Invoice_{prefix}_" + '_'.join(str(x) for x in params_values.values()) + '.pdf')
    filename.write_bytes(response.content)

papers = ['TD', 'TP', 'FI', 'RV', 'PS']
for paper in papers:
  get_invoices(paper)



TD <Response [200]>
TP <Response [204]>
No data for TP
FI <Response [200]>
RV <Response [200]>
PS <Response [204]>
No data for PS


In [15]:
# Process all pdfs to txt
%%bash
for f in *.pdf
do
 pdf2txt.py -o $f.txt $f
done